# 1. Process Class Implementation

In [15]:
from typing import List
import heapq
import random as rand
import pandas as pd

class Process:
    def __init__(self, id: int, burst_time: int, priority: int, arr_time: int , waiting_time : int = 0, cpu_time_acquired : int  = 0, turnaround_time : int = 0) -> None:
        
        self.id = id
        self.burst_time = burst_time
        self.priority = priority
        self.arrival_time = arr_time

        # waiting time is the time between submission to system and starting execution 

        
        self.waiting_time = 0
        self.cpu_time_acquired = 0
        self.turnaround_time = 0
        

        # this would be relevant for preemptive algorithms
        self.last_running_time = 0
        self.max_waiting_time = 0
        self.age = 0
        


    

    @staticmethod
    def generate_processes(file_name :str, number_of_processes : int ,  max_burst :int , min_burst:int, max_arrival_time:int) -> []:
        file = open(file_name, 'w')
        file.write("id,burst_time,priority,arrival_time,turnaround_time,cpu_time_acquired,waiting_time" + '\n')
        processes = []
        for i in range(number_of_processes):
            
            id = i+1
            priority = rand.randint(-20, 19)
            burst_time = rand.randint(min_burst, max_burst)
            arrival_time = rand.randint(0,max_arrival_time)
            
            process = Process(id, burst_time, priority, arrival_time)
            processes.append(process)
            file.write(str(process) + '\n')

        file.close()
            
        return processes
    
    @staticmethod
    def load_from_csv(file_name : str) -> []:
        processes =  []
        df = pd.read_csv(file_name)
        for _,row in df.iterrows() :
            process = Process(row["id"], row["burst_time"],row["priority"], row["arrival_time"], row["turnaround_time"] , row["cpu_time_acquired"], row["waiting_time"])
            processes.append(process)

        return processes
    

    @staticmethod 
    def save_processes_csv(file_name : str , processes : []) -> None :
        file = open(file_name , 'w')
        file.write("id,burst_time,priority,arrival_time,turnaround_time,cpu_time_acquired,waiting_time" + '\n')
        for process in processes : 
            file.write(str(process) + '\n')
        

    def __lt__(self, other):
        if self.priority != other.priority:
            return self.priority < other.priority
        else:
            return self.id < other.id


    def __str__(self):
        return str(self.id) + ',' + str(self.burst_time) + ',' + str(self.priority) + ',' + str(self.arrival_time) + ',' + str(self.turnaround_time) + ',' + str(self.cpu_time_acquired) +',' + str(self.waiting_time)
    

    def display(self):
        return f"Process {self.id} with burst time {self.burst_time} and priority {self.priority} and arrival time {self.arrival_time} and waiting time {self.waiting_time} and turnaround time {self.turnaround_time}"


In [16]:
### we will generate a list of processes that will later be used for testing the algorithms
processes = Process.generate_processes("mahmoud.csv", 1000, 20, 1, 100)

# 2. Batch Scheduling Algorithms

## First come first served

In [17]:
# First_come_First_Served
def First_come_First_Served_2(Processes: List[Process]):
    
    # write the output to a csv file
    file = open("execution.csv", 'w')

    Processes.sort(key=lambda x: (x.arrival_time, x.id))
    
    # Calculate waiting time for each process
    current_time = Processes[0].arrival_time



    for process in Processes:

        
        if process.arrival_time > current_time:
            current_time = process.arrival_time

        file.write(str(process.id) + ',' + str(current_time) + ',')

        
        process.waiting_time = current_time - process.arrival_time
        process.turnaround_time = process.waiting_time + process.burst_time
        current_time += process.burst_time


        file.write(str(current_time) + '\n')

    Processes.sort(key=lambda x: x.id)
    return Processes, current_time

In [18]:
# load the processes from the csv file
processes = Process.load_from_csv("mahmoud.csv")

# run the priority round robin algorithm
processes, current_time = First_come_First_Served_2(processes)

In [19]:
for i in processes:
    print(i.display())

Process 1 with burst time 8 and priority 4 and arrival time 71 and waiting time 7008 and turnaround time 7016
Process 2 with burst time 15 and priority -9 and arrival time 73 and waiting time 7199 and turnaround time 7214
Process 3 with burst time 19 and priority 18 and arrival time 69 and waiting time 6759 and turnaround time 6778
Process 4 with burst time 4 and priority 11 and arrival time 80 and waiting time 7984 and turnaround time 7988
Process 5 with burst time 10 and priority -6 and arrival time 24 and waiting time 2110 and turnaround time 2120
Process 6 with burst time 2 and priority -5 and arrival time 71 and waiting time 7016 and turnaround time 7018
Process 7 with burst time 5 and priority -8 and arrival time 53 and waiting time 5126 and turnaround time 5131
Process 8 with burst time 7 and priority -8 and arrival time 38 and waiting time 3609 and turnaround time 3616
Process 9 with burst time 8 and priority 12 and arrival time 58 and waiting time 5569 and turnaround time 5577

In [20]:
# the average turnaround time
print("The average turnaround time is: ", sum([i.turnaround_time for i in processes]) / len(processes))

The average turnaround time is:  5132.398


In [21]:
# the average waiting time
print("The average waiting time is: ", sum([i.waiting_time for i in processes]) / len(processes))

The average waiting time is:  5122.088


## Shortest job first

In [9]:
# shortest job first 
def Shortest_Job_First_2(Processes: List[Process]): 
    
    # we will use a heap to store the processes with the highest priority at the top
    # and we will push processes in the heap as they arrive and pop the process with the highest priority

    file = open("execution.csv", 'w')

    heap = []
    heapq.heapify(heap)

    # Sort by arrival time
    Processes.sort(key=lambda x: x.arrival_time)

    
    current_process_idx = 0
    current_time = Processes[0].arrival_time

    # add all processes that have arrived to the heap
    while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
        heapq.heappush(heap, (Processes[current_process_idx].burst_time, Processes[current_process_idx]))
        current_process_idx += 1
    


    # Now while the queue is not empty we will take the element with the highest priority and calculate the waiting time and current time after the process finishes
    # and then we will push all processes that have arrived to the heap and repeat the process

    
    while heap:


        
        process = heapq.heappop(heap)

        file.write(str(process[1].id) + ',' + str(current_time) + ',')


        process[1].waiting_time = current_time - process[1].arrival_time
        process[1].turnaround_time = process[1].waiting_time + process[1].burst_time
        current_time += process[1].burst_time

        file.write(str(current_time) + '\n')

        while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
            heapq.heappush(heap, (Processes[current_process_idx].burst_time, Processes[current_process_idx]))
            current_process_idx += 1
        
        # now the heap might be empty but there are still processes that have not arrived yet
        # we will update the current time to the arrival time of the next process

        if current_process_idx < len(Processes) and heap == []:
            current_time = Processes[current_process_idx].arrival_time

        # now add the process to the heap
        while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
            heapq.heappush(heap, (Processes[current_process_idx].burst_time, Processes[current_process_idx]))
            current_process_idx += 1

    Processes.sort(key=lambda x: x.id)
    return Processes, current_time

In [22]:
# load the processes from the csv file
processes = Process.load_from_csv("mahmoud.csv")

In [23]:
# run the priority round robin algorithm
processes, current_time = Shortest_Job_First_2(processes)

In [27]:
# display the five first processes
counter = 0
for i in processes:
    print(i.display())
    counter += 1
    if counter == 5:
        break

Process 1 with burst time 8 and priority 4 and arrival time 71 and waiting time 1537 and turnaround time 1545
Process 2 with burst time 15 and priority -9 and arrival time 73 and waiting time 5150 and turnaround time 5165
Process 3 with burst time 19 and priority 18 and arrival time 69 and waiting time 9223 and turnaround time 9242
Process 4 with burst time 4 and priority 11 and arrival time 80 and waiting time 412 and turnaround time 416
Process 5 with burst time 10 and priority -6 and arrival time 24 and waiting time 2468 and turnaround time 2478
Process 6 with burst time 2 and priority -5 and arrival time 71 and waiting time 37 and turnaround time 39
Process 7 with burst time 5 and priority -8 and arrival time 53 and waiting time 608 and turnaround time 613
Process 8 with burst time 7 and priority -8 and arrival time 38 and waiting time 1118 and turnaround time 1125
Process 9 with burst time 8 and priority 12 and arrival time 58 and waiting time 1590 and turnaround time 1598
Process

In [25]:
# the average turnaround time
print("The average turnaround time is: ", sum([i.turnaround_time for i in processes]) / len(processes))

The average turnaround time is:  3409.484


In [26]:
# the average waiting time
print("The average waiting time is: ", sum([i.waiting_time for i in processes]) / len(processes))

The average waiting time is:  3399.174


## Priority scheduling for Batch jobs

In [219]:
## Priority Scheduling
def Priority_2(Processes: List[Process]):
    
    # we will use a heap to store the processes with the highest priority at the top
    # and we will push processes in the heap as they arrive and pop the process with the highest priority

    heap = []
    heapq.heapify(heap)

    file = open("execution.csv", 'w')

    # Sort by arrival time
    Processes.sort(key=lambda x: x.arrival_time)

    
    current_process_idx = 0
    current_time = Processes[0].arrival_time

    # add all processes that have arrived to the heap
    while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
        heapq.heappush(heap, (Processes[current_process_idx].priority, Processes[current_process_idx]))
        current_process_idx += 1
    


    # Now while the queue is not empty we will take the element with the highest priority and calculate the waiting time and current time after the process finishes
    # and then we will push all processes that have arrived to the heap and repeat the process

    
    while heap:
        # print('Trated process:', heap[0][1].id)
        
        process = heapq.heappop(heap)

        file.write(str(process[1].id) + ',' + str(current_time) + ',')
        process[1].waiting_time = current_time - process[1].arrival_time
        process[1].turnaround_time = process[1].waiting_time + process[1].burst_time
        current_time += process[1].burst_time

        file.write(str(current_time) + '\n')

        while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
            heapq.heappush(heap, (Processes[current_process_idx].priority, Processes[current_process_idx]))
            current_process_idx += 1
        
        # now the heap might be empty but there are still processes that have not arrived yet
        # we will update the current time to the arrival time of the next process

        if current_process_idx < len(Processes) and heap == []:
            current_time = Processes[current_process_idx].arrival_time

        # now add the process to the heap
        while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
            heapq.heappush(heap, (Processes[current_process_idx].priority, Processes[current_process_idx]))
            current_process_idx += 1
            
    Processes.sort(key=lambda x: x.id)
    return Processes, current_time

In [223]:
# generate random processes
processes = Process.generate_processes("mahmoud.csv", 10, 20, 5, 10)

# load the processes from the csv file
processes = Process.load_from_csv("mahmoud.csv")

In [224]:
# run the priority round robin algorithm
processes, current_time = Priority_2(processes)

# 3.Interactive systems Scheduling Algorithms

## Priority Round Robin Algorithm

In [61]:

from collections import deque

def Priority_round_robin_2(processes : List[Process], quantum : int, context_switching_time : int) -> List[Process]:

    # this time we will write to a csv file which process is being executed and at what time
    file = open("execution.csv", 'w')

    processes.sort(key=lambda x: x.arrival_time)

    # the priority classes range from -20 to 19
    queue_list = [deque() for _ in range(40)]

    current_time = processes[0].arrival_time
    process_index = 0
    finished_processes = 0
    n = len(processes)

    prev = -1


    while process_index < n and processes[process_index].arrival_time == current_time:
        queue_list[processes[process_index].priority + 20].append(processes[process_index])
        process_index += 1        


    while finished_processes < n:

        # dipslay a progression bar here and delete it ater each progress
        # ...
        
        progress = finished_processes / n
        bar_length = 30
        bar = '=' * int(progress * bar_length) + '>' + '-' * (bar_length - int(progress * bar_length))
        print(f"\rProgress: [{bar}] {progress * 100:.2f}%", end='', flush=True)




        found = False 
        for i in range(40):
            if len(queue_list[i]) > 0:
                found = True
                process = queue_list[i].popleft()

                if prev == -1:
                    prev = process.id

                
                if context_switching_time != 0 and prev != process.id:
                        file.write("Context_Switch," + str(current_time) + ',')
                        current_time += context_switching_time
                        file.write(str(current_time) + '\n')



                file.write(str(process.id) + ',' + str(current_time) + ',')



                # print("-----> Operating on process ", process.id, " with priority ", process.priority, " at time ", current_time)

                if process.cpu_time_acquired == 0:
                    process.waiting_time = current_time - process.arrival_time

                
                if process.burst_time - process.cpu_time_acquired <= quantum:
                    current_time += process.burst_time - process.cpu_time_acquired
                    process.cpu_time_acquired = process.burst_time
                    process.turnaround_time = current_time - process.arrival_time
                    finished_processes += 1
                    file.write(str(current_time) + '\n')


                    while process_index < n and processes[process_index].arrival_time <= current_time:
                        # print("Added process ", processes[process_index].id, " to queue ")
                        queue_list[processes[process_index].priority + 20].append(processes[process_index])
                        process_index += 1
                    


                else:
                    current_time += quantum
                    process.cpu_time_acquired += quantum
                    file.write(str(current_time) + '\n')


                    while process_index < n and processes[process_index].arrival_time <= current_time:
                        # print("Added process ", processes[process_index].id, " to queue ")
                        queue_list[processes[process_index].priority + 20].append(processes[process_index])
                        process_index += 1
                    queue_list[i].append(process)

                
                prev = process.id
                # print("-----> Finished process ", process.id, " at time ", current_time)
                break

        if not found:
            current_time = processes[process_index].arrival_time
            while process_index < n and processes[process_index].arrival_time == current_time:
                queue_list[processes[process_index].priority + 20].append(processes[process_index])
                process_index += 1

    file.close()

    # sort the processes based on their id
    processes.sort(key=lambda x: x.id)
    return processes, current_time


In [62]:
# generate random processes
processes = Process.generate_processes("mahmoud.csv", 100, 20, 5, 10)

In [63]:
# load the processes from the csv file
processes = Process.load_from_csv("mahmoud.csv")

# run the priority round robin algorithm
processes, current_time = Priority_round_robin_2(processes, 2, 1)


Progress: [=============================>-] 99.00%

## Round Robin

In [68]:
from collections import deque

# Round robin Algorithm
def Round_Robin_2(Processes: List[Process], quantum: int, context_switching_time: int):
    
    file = open("execution.csv", 'w')
    file.write("id,arrival_time,finish_time" + '\n')
    Processes.sort(key=lambda x: x.arrival_time)

    current_time = Processes[0].arrival_time
    finished_processes = 0

    queue = deque()


    # current_process_idx is the index of the next process that will arrive
    current_process_idx = 0


    while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
        queue.append(Processes[current_process_idx])
        current_process_idx += 1

    prev = -1

    while finished_processes < len(Processes):

        # first we do a context switch 
        

        # we pick the first process in the queue and execute it for the quantum time
        # if the process finishes before the quantum time we will update the current time and the process

        if queue:
            process = queue.popleft()

            if prev == -1:
                prev = process.id

            if context_switching_time != 0 and prev != process.id:
                file.write("Context_Switch," + str(current_time) + ',')
                current_time += context_switching_time
                file.write(str(current_time) + '\n')

            file.write(str(process.id) + ',' + str(current_time) + ',')

            # set the waiting time of the process
            if process.cpu_time_acquired == 0:
                process.waiting_time = current_time - process.arrival_time

            #print("runnnig process", process.id)
            if process.burst_time - process.cpu_time_acquired <= quantum:
                process.cpu_time_acquired = process.burst_time
                current_time += process.burst_time - process.cpu_time_acquired
                process.turnaround_time = current_time - process.arrival_time
                finished_processes += 1

                while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
                    queue.append(Processes[current_process_idx])
                    current_process_idx += 1

                
            
            else:
                process.cpu_time_acquired += quantum
                current_time += quantum

                while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
                    queue.append(Processes[current_process_idx])
                    current_process_idx += 1
                
                queue.append(process)

            prev = process.id
            file.write(str(current_time) + '\n')

                
        else:
            current_time = Processes[current_process_idx].arrival_time

            while current_process_idx < len(Processes) and Processes[current_process_idx].arrival_time <= current_time:
                    queue.append(Processes[current_process_idx])
                    current_process_idx += 1

        
    Processes.sort(key=lambda x: x.id)
    return Processes, current_time

In [69]:
# generate random processes
processes = Process.generate_processes("mahmoud.csv", 100, 20, 5, 10)

# load the processes from the csv file
processes = Process.load_from_csv("mahmoud.csv")

In [70]:
# run the RR algorithm
processes, current_time = Round_Robin_2(processes, 2, 1)

## Round Robin with Aging

In [ ]:
def round_robin_with_aging(processes: List[Process], quantum: int, context_switching_time: int) -> List[Process]:
    # this time we will write to a csv file which process is being executed and at what time
    file = open("execution.csv", 'w')

    processes.sort(key=lambda x: x.arrival_time)

    queue = []

    current_time = processes[0].arrival_time
    process_index = 0
    finished_processes = 0
    n = len(processes)

    prev = -1

    while process_index < n and processes[process_index].arrival_time == current_time:
        heapq.heappush(queue, (processes[process_index].age, processes[process_index]))
        process_index += 1

    while finished_processes < n:

        # dipslay a progression bar here and delete it ater each progress
        # ...

        progress = finished_processes / n
        bar_length = 30
        bar = '=' * int(progress * bar_length) + '>' + '-' * (bar_length - int(progress * bar_length))
        print(f"\rProgress: [{bar}] {progress * 100:.2f}%", end='', flush=True)

        if len(queue) == 0:
            current_time = processes[process_index].arrival_time
            while process_index < n and processes[process_index].arrival_time == current_time:
                heapq.heappush(queue, (processes[process_index].age, processes[process_index]))
                process_index += 1

        age, process = heapq.heappop(queue)
        # print("Operating on process ", process.id, " with age ", process.age, " at time ", current_time)
        if prev == -1:
            prev = process.id

        if context_switching_time != 0 and prev != process.id:
            file.write("Context_Switch," + str(current_time) + ',')
            current_time += context_switching_time
            file.write(str(current_time) + '\n')

        file.write(str(process.id) + ',' + str(current_time) + ',')

        if process.burst_time - process.cpu_time_acquired <= quantum:
            current_time += process.burst_time - process.cpu_time_acquired
            process.cpu_time_acquired = process.burst_time
            process.turnaround_time = current_time - process.arrival_time

            process.max_waiting_time = max(process.max_waiting_time, current_time - process.last_running_time)
            process.last_running_time = current_time

            process.total_waiting_time = process.turnaround_time - process.burst_time
            finished_processes += 1
            file.write(str(current_time) + '\n')

            while process_index < n and processes[process_index].arrival_time <= current_time:
                heapq.heappush(queue, (processes[process_index].age, processes[process_index]))
                process_index += 1

        else:
            current_time += quantum
            process.cpu_time_acquired += quantum
            file.write(str(current_time) + '\n')

            process.max_waiting_time = max(process.max_waiting_time, current_time - process.last_running_time)
            process.last_running_time = current_time

            while process_index < n and processes[process_index].arrival_time <= current_time:
                heapq.heappush(queue, (processes[process_index].age, processes[process_index]))
                process_index += 1

            process.age += (process.priority + 21)
            heapq.heappush(queue, (process.age, process))

        prev = process.id

    file.close()

    # compute the waiting time for each process
    for process in processes:
        process.waiting_time = process.turnaround_time - process.burst_time

    # sort the processes based on their id
    processes.sort(key=lambda x: x.id)

    #  write into a csv file.
    file = open("SpecialFile.csv", 'w')


    for process in processes:
        file.write(str(process.max_waiting_time) + ',')
    file.write('\n')
    for process in processes:
        file.write(str(process.waiting_time) + ',')
    file.write('\n')
    for process in processes:
        file.write(str(process.turnaround_time) + ',')

    file.close()

    
    return processes, current_time

In [ ]:
# generate random processes
processes = Process.generate_processes("mahmoud.csv", 100, 20, 5, 10)

# load the processes from the csv file
processes = Process.load_from_csv("mahmoud.csv")

In [ ]:
# run the RR algorithm
processes, current_time = round_robin_with_aging(processes, 2, 1)